In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 14.3MB/s 
     |████████████████████████████████| 3.3MB 29.8MB/s 
     |████████████████████████████████| 901kB 49.8MB/s 
     |████████████████████████████████| 1.2MB 14.6MB/s 


In [ ]:
import torch
#torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

import json
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
print(device)
# print(torch.cuda.get_device_name(0))

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/CS182-Spring2020-NLP-Project/"

Mounted at /content/drive


In [ ]:
data = []
with open(root_folder + 'dataset/test_data.jsonl', 'r') as file:
    data = [json.loads(jline) for jline in file.read().splitlines()]

In [ ]:
labels = [int(review["stars"])-1 for review in data]

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False, remove_space=False)
data = [tokenizer.tokenize(review["text"] + " [SEP] [CLS]") for review in data]
print ("Tokenize the first sentence:")
print (data[0])


Tokenize the first sentence:
['▁', 'i', '▁great', '▁experience', '▁', 'ik', 'on', '▁', 'i', '▁went', '▁washing', '▁cutting', '▁bloody', 'ing', '▁curling', '▁hair', '▁friends', '▁birthday', '▁that', '▁', 'i', '▁paid', '▁total', '▁70', '▁tax', '▁they', '▁specific', '▁flat', '▁rate', '▁hair', 'dress', 'er', '▁salon', '▁hair', 'dress', 'er', '▁whose', '▁name', '▁', 'i', '▁believe', '▁mi', 'cha', 'el', '▁really', '▁best', '▁make', '▁sure', '▁everything', '▁looked', '▁', 'i', '▁wanted', '▁it', '▁too', '▁bad', '▁phone', '▁died', '▁', 'i', '▁left', '▁', 'i', '▁take', '▁picture', '▁amazing', '▁hair', '▁looked', '▁', 'i', '▁remember', '▁getting', '▁many', '▁compliment', 's', '▁hair', '▁', 'loo', 'ong', '▁time', '▁', 'i', '▁definitely', '▁go', '▁back', '▁next', '▁cut', '▁whenever', '▁', 'i', '▁need', '▁get', '▁hair', '▁done', '▁nicely', '▁really', '▁worth', '▁money', '▁', 'i', '▁also', '▁liked', '▁friendly', '▁service', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']']


In [ ]:
MAX_LEN = 512

In [ ]:
data = [tokenizer.convert_tokens_to_ids(x) for x in data]

In [ ]:
data = pad_sequences(data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
attention_masks = []

for seq in data:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
data = torch.tensor(data)
labels = torch.tensor(labels)
attention_masks = torch.tensor(attention_masks)

In [ ]:
batch_size = 1

train_data = TensorDataset(data, attention_masks, labels)
train_sampler = RandomSampler(train_data)
prediction_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=5)
model = XLNetForSequenceClassification.from_pretrained("/content/drive/My Drive/XLNet_latest", num_labels=5)
model.cuda()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [ ]:
model.eval()

total_diff = 0
correct = 0
count = 0

predictions , true_labels = [], []

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  
  with torch.no_grad():    
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  predictions.append(logits)
  true_labels.append(label_ids)
  
  for i in range(len(logits)):
    pred = np.argmax(logits[i])
    total_diff += abs(pred - int(label_ids[i]))
    correct += 1 if pred == int(label_ids[i]) else 0
    count += 1
  print("Predicted:", pred + 1,
      "\tActual:", int(label_ids[i]) + 1,
      "\tDiff:", total_diff,
      "\tAccuracy:", correct / count)

Streaming output truncated to the last 5000 lines.
Predicted: 5 	Actual: 5 	Diff: 11612 	Accuracy: 0.8059926797493745
Predicted: 1 	Actual: 1 	Diff: 11612 	Accuracy: 0.8059966914805624
Predicted: 5 	Actual: 5 	Diff: 11612 	Accuracy: 0.8060007030458427
Predicted: 5 	Actual: 5 	Diff: 11612 	Accuracy: 0.8060047144452256
Predicted: 5 	Actual: 5 	Diff: 11612 	Accuracy: 0.8060087256787213
Predicted: 5 	Actual: 4 	Diff: 11613 	Accuracy: 0.8059920602100736
Predicted: 1 	Actual: 1 	Diff: 11613 	Accuracy: 0.8059960715393363
Predicted: 1 	Actual: 1 	Diff: 11613 	Accuracy: 0.806000082702725
Predicted: 4 	Actual: 5 	Diff: 11614 	Accuracy: 0.8059834184464614
Predicted: 5 	Actual: 5 	Diff: 11614 	Accuracy: 0.8059874297055905
Predicted: 5 	Actual: 4 	Diff: 11615 	Accuracy: 0.8059707663999669
Predicted: 1 	Actual: 2 	Diff: 11616 	Accuracy: 0.8059541037833368
Predicted: 1 	Actual: 1 	Diff: 11616 	Accuracy: 0.805958115399723
Predicted: 1 	Actual: 1 	Diff: 11616 	Accuracy: 0.8059621268502439
Predicted: 1 